##### Copyright 2019 The TensorFlow Authors.

# Save and load models

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.

### Options

There are different ways to save TensorFlow models—depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow. For other approaches, see the TensorFlow  [Save and Restore](https://www.tensorflow.org/guide/saved_model) guide or [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving).

## Setup

### Installs and imports

Install and import TensorFlow and dependencies:

In [1]:
# !pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


### Get an example dataset

To demonstrate how to save and load weights, you'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). To speed up these runs, use the first 1000 examples:  

#### numpy中reshape函数的三种常见相关用法  
reshape(1,-1)转化成1行：  
reshape(2,-1)转换成两行：  
reshape(-1,1)转换成1列：  
reshape(-1,2)转化成两列  

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

# 降维 1000,28,28(三维) -> 1000,784（二维）
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)          # mnist 是0-9 的数字识别，所以输入是10管道，然后看哪个管道的数字大，就是几率高
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

In [5]:
# print(os.path.abspath(checkpoint_path))

You can use a trained model without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.3167 - accuracy: 0.6083
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.2545 - accuracy: 0.6250 - val_loss: 0.7581 - val_accuracy: 0.7560
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4357 - accuracy: 0.8761
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 394us/sample - loss: 0.4410 - accuracy: 0.8760 - val_loss: 0.5416 - val_accuracy: 0.8380
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2919 - accuracy: 0.9278
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 388us/sample - loss: 0.2925 - accuracy: 0.9290 - val_loss: 0.4991 - val_accuracy: 0.8450
Epoch 4/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2135 - accuracy: 0.9483
Epoch 0

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [7]:
!ls {checkpoint_dir}

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


Create a new, untrained model. When restoring a model from weights-only, you must have a model with the same architecture as the original model. Since it's the same model architecture, you can share weights despite that it's a different *instance* of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):  
创建一个新的未经训练的模型。仅恢复模型的权重时，必须具有与原始模型具有相同网络结构的模型。由于模型具有相同的结构，您可以共享权重，尽管它是模型的不同实例。 现在重建一个新的未经训练的模型，并在测试集上进行评估。未经训练的模型将在机会水平（chance levels）上执行（准确度约为10％）：

In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.4449 - accuracy: 0.0620
Untrained model, accuracy:  6.20%


Then load the weights from the checkpoint and re-evaluate:  
然后从 checkpoint 加载权重并重新评估：

In [9]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4715 - accuracy: 0.8650
Restored model, accuracy: 86.50%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [10]:
# Include the epoch in the file name (uses `str.format`)
# 在文件名中包含 epoch (使用 `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"                  # 格式变了
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)                    # 之前没有 

# Create a new model instance
# 创建一个新的模型实例
model = create_model()

# Save the weights using the `checkpoint_path` format
# 使用 `checkpoint_path` 格式保存权重
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
# 使用新的回调*训练*模型
model.fit(train_images, 
          train_labels,
          epochs=50,                     # 之前是epochs=10
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


Now, look at the resulting checkpoints and choose the latest one:

In [11]:
!ls {checkpoint_dir}

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

Note: the default tensorflow format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [13]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
# 等效于 model.load_weights('training_2\\cp-0050.ckpt')
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4690 - accuracy: 0.8760
Restored model, accuracy: 87.60%


## What are these files?

The above code stores the weights to a collection of [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in a which shard.

If you are only training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, `tf.keras`—and `save_weights` in particular—uses the TensorFlow [checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension (saving in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) with a `.h5` extension is covered in the [Save and serialize models](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) guide):

In [14]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4690 - accuracy: 0.8760
Restored model, accuracy: 87.60%


## Save the entire model

Call [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Entire model can be saved in two different file formats (`SavedModel` and `HDF5`). It is to be noted that TensorFlow `SavedModel` format is the default file format in TF2.x. However, model can be saved in `HDF5` format. More details on saving entire model in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/convert/python_api#converting_a_savedmodel_), [HDF5](https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_))

\*Custom objects (e.g. subclassed models or layers) require special attention when saving and loading. See the **Saving custom objects** section below 

### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The [SavedModel guide](https://www.tensorflow.org/guide/saved_model) goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to saving and restoring the model.

In [28]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 598us/sample - loss: 1.1355 - accuracy: 0.6780
Epoch 2/5
1000/1000 [==============================] - 0s 187us/sample - loss: 0.4119 - accuracy: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 222us/sample - loss: 0.2909 - accuracy: 0.9220
Epoch 4/5
1000/1000 [==============================] - 0s 204us/sample - loss: 0.2049 - accuracy: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 223us/sample - loss: 0.1477 - accuracy: 0.9670


The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory:

In [25]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


Reload a fresh Keras model from the saved model:

In [30]:
new_model = tf.keras.models.load_model('saved_model/my_model.h5')

# Check its architecture
new_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [32]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

a = new_model.predict(test_images)
print(a.shape)

1000/1 - 0s - loss: 0.5697 - accuracy: 0.8540
Restored model, accuracy: 85.40%
(1000, 10)


In [42]:
import numpy as np
# np.argmax(predictions[0])
print(np.argmax(a[1]),test_labels[1],a[1])

2 2 [-0.81889784 -1.0836033   5.2391744   1.9389149  -6.041071    2.1121325
  3.4519668  -7.9149656   0.8982452  -3.4417427 ]


### HDF5 format

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [43]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 628us/sample - loss: 1.1712 - accuracy: 0.6660
Epoch 2/5
1000/1000 [==============================] - 0s 199us/sample - loss: 0.4440 - accuracy: 0.8760
Epoch 3/5
1000/1000 [==============================] - 0s 215us/sample - loss: 0.2906 - accuracy: 0.9220
Epoch 4/5
1000/1000 [==============================] - 0s 201us/sample - loss: 0.2103 - accuracy: 0.9580
Epoch 5/5
1000/1000 [==============================] - 0s 194us/sample - loss: 0.1654 - accuracy: 0.9630


Now, recreate the model from that file:

In [44]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [45]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 - 0s - loss: 0.5287 - accuracy: 0.8660
Restored model, accuracy: 86.60%


Keras saves models by inspecting the architecture. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration(what you passed to compile)
* The optimizer and its state, if any (this enables you to restart training where you left)

Keras is not able to save the `v1.x` optimizers (from `tf.compat.v1.train`) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.


### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the orginal code.

To save custom objects to HDF5, you must do the following:

1. Define a `get_config` method in your object, and optionally a `from_config` classmethod.
  * `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
  * `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs (`return cls(**config)`).
2. Pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

See the [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) tutorial for examples of custom objects and `get_config`.
